# Percolación Ejercicio 1
***
## Grupo:
1. Adrián Bedón
1. Merlo José Miguel
1. Moreta Andrés
1. Ocaña Dennis
1. Ramos Xavier
***

## Qué es la percolación?
Percolación hace referencia al paso de fluidos a travez de materiales porosos. Un ejemplo de percolación es la filtracion<br>
<br>
<div>
<img src="https://cdn0.ecologiaverde.com/es/posts/3/2/1/como_hacer_un_filtro_de_agua_casero_para_beber_1123_orig.jpg" width="500"/>
</div>

***

## Simulación de Montecarlo para estudiar un fenómeno natural conocido como percolación

Para el proposito de estas simulaciones se definirá un sistema de `n * n` a modo de cuaadrícula. Dentro de esta cada cuadrante puede estar bloqueado o disponible y los cuadrantes disponibles son inicializados _vacíos_. Un sitio _lleno_ es un sitio disponible que se conecta a un sitio aledaño que también está _diponible_ siendo estos: arriba, abajo, izquierda y/o derecha).<br>
<br>
El sistema se dirá que percola si es que hay un sitio disponible _completo_ esto significando que esta conectado de un extremo a otro de la cuadrícula.<br>
<br>
<div>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-yes.png" width="300"/>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-no.png" width="300"/>
</div>

***

## Percolación vertical
Dada una matriz que representa los sitios disponibles, debemos determinar si este sistema percola. Empezando com percolación vertical siendo este el más simple ya que solo buscará percolación directamente en el prano vertical
<br>
<br>
<div>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-vertically-yes.png" width="300"/>
<img src="https://introcs.cs.princeton.edu/python/24percolation/images/percolates-vertically-no.png" width="300"/>
</div>

***

In [4]:
## Importamos librerias necesarias
#import numpy as np
import numpy as np #importamos numpy para utilizar linspace
import numba as nb #se importa numba para aceleracion por GPU en diversas funciones
from numba import cuda #para aceleracion por GPU en diversas funciones
from numba import jit #para aceleracion por GPU en diversas funciones
from numba import njit #para aceleracion por GPU en diversas funciones

In [12]:
## Arma un array segun las probabilidades que se indique para las celdas disponibles
@jit
def armarArray(n, probabilidadDisponible):
    array = np.random.binomial(1, probabilidadDisponible, size=(n,n))
    return array

In [13]:
@jit
def flowVertical(array):
    n = len(array)
    ArrayCamino = np.zeros((n,n),dtype=int)
    for j in range(n):
        _flowVertical(array, ArrayCamino, 0, j)
    return ArrayCamino

In [14]:
## Arma un funcion recursiva para armar un array aparte que solo tendra los registros de las
## celdas disponibles que esten conectadas entre si
@jit
def _flowVertical(isOpen, isFull, i, j):
    n = len(isFull)
    if (i < 0) or (i >= n):
        return
    if (j < 0) or (j >= n):
        return
    if (isOpen[i][j] == 0):
        return
    if (isFull[i][j] == 1):
        return
    isFull[i][j] = 1
    _flowVertical(isOpen, isFull, i+1, j)  ## Abajo

In [15]:
## Ejecuta los segmentos recursivos del codigo y detecta si hay un 1 en la fila final
## ya que al haberlo, significa que si percola
@jit
def percolaVertical(isOpen):
    
    ## crea la matriz de 1 que estan juntos
    isFull = flowVertical(isOpen)
    
    
    ## verifica si hay algun 1 en la fila final ya que si es asi, el sistema si percola
    n = len(isFull)
    for j in range(n):
        if (isFull[n-1][j]):
            return True
    return False


In [18]:
## definimos una funcion que tomara n siendo esta la dimension de la matriz
## y p siendo la probabilidad de que la celda este disponible
## esta version incluye un print por propositos de demostración

def mainVerticalPrint(n,p):
    array = armarArray(n,p)
    print(array)
    print(percolaVertical(array))
    
mainVerticalPrint(2,0.5)

[[1 1]
 [0 1]]
True


***

### Pruebas para percolacion vertical con 𝑝∈{0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9}. 

In [22]:
## definimos una funcion que tomara n siendo esta la dimension de la matriz
## y p siendo la probabilidad de que la celda este disponible

def mainVertical(n,p):
    array = armarArray(n,p)
    return percolaVertical(array)

In [23]:
## para 0.1

j=10000000
contador=0
p=0.1
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 199634 veces
Esto resulta en una probabilidad de 0.0199634
Comparado con la probabilidad por fórmula la cual es 0.019900000000000004


In [24]:
## para 0.2

j=10000000
contador=0
p=0.2
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 784706 veces
Esto resulta en una probabilidad de 0.0784706
Comparado con la probabilidad por fórmula la cual es 0.07840000000000001


In [25]:
## para 0.3

j=10000000
contador=0
p=0.3
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 1716629 veces
Esto resulta en una probabilidad de 0.1716629
Comparado con la probabilidad por fórmula la cual es 0.1719


In [26]:
## para 0.4

j=10000000
contador=0
p=0.4
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 2943988 veces
Esto resulta en una probabilidad de 0.2943988
Comparado con la probabilidad por fórmula la cual es 0.29440000000000005


In [27]:
## para 0.5

j=10000000
contador=0
p=0.5
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 4377055 veces
Esto resulta en una probabilidad de 0.4377055
Comparado con la probabilidad por fórmula la cual es 0.4375


In [28]:
## para 0.6

j=10000000
contador=0
p=0.6
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 5903409 veces
Esto resulta en una probabilidad de 0.5903409
Comparado con la probabilidad por fórmula la cual es 0.5904


In [29]:
## para 0.7

j=10000000
contador=0
p=0.7
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 7400449 veces
Esto resulta en una probabilidad de 0.7400449
Comparado con la probabilidad por fórmula la cual es 0.7398999999999999


In [30]:
## para 0.8

j=10000000
contador=0
p=0.8
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 8704684 veces
Esto resulta en una probabilidad de 0.8704684
Comparado con la probabilidad por fórmula la cual es 0.8704000000000001


In [31]:
## para 0.9

j=10000000
contador=0
p=0.9
for i in range (j):
    if mainVertical(2,p):
        contador=contador+1
print("Percoló", contador, "veces")
print("Esto resulta en una probabilidad de", contador/j)
print("Comparado con la probabilidad por fórmula la cual es", p**2*(2-p**2))

Percoló 9639805 veces
Esto resulta en una probabilidad de 0.9639805
Comparado con la probabilidad por fórmula la cual es 0.9639


***

In [ ]:
## Arma un funcion recursiva para armar un array aparte que solo tendra los registros de las
## celdas disponibles que esten conectadas entre si
def _flow(isOpen, isFull, i, j):
    n = len(isFull)
    if (i < 0) or (i >= n):
        return
    if (j < 0) or (j >= n):
        return
    if (isOpen[i][j] == 0):
        return
    if (isFull[i][j] == 1):
        return
    isFull[i][j] = 1
    _flow(isOpen, isFull, i+1, j  )  ## Abajo
    _flow(isOpen, isFull, i  , j+1)  ## Derecha
    _flow(isOpen, isFull, i  , j-1)  ## Izquierda
    _flow(isOpen, isFull, i-1, j  )  ## Arriba

In [ ]:
## Crea un array nuevo en el cual se guardaran las celdas disponibles que esten conectadas
## a las demas y las detecta
def flow(isOpen):
    n = len(isOpen)
    isFull = np.zeros((n,n),dtype=int)
    for j in range(n):
        _flow(isOpen, isFull, 0, j)
    print(isFull)
    return isFull

In [ ]:
## Ejecuta los segmentos recursivos del codigo y detecta si hay un 1 en la fila final
## ya que al haberlo, significa que si percola
def percola(isOpen):
    
    ## crea la matriz de 1 que estan juntos
    isFull = flow(isOpen)
    
    
    ## verifica si hay algun 1 en la fila final ya que si es asi, el sistema si percola
    n = len(isFull)
    for j in range(n):
        if (isFull[n-1][j]):
            return True
    return False


In [ ]:
## Arma un funcion recursiva para armar un array aparte que solo tendra los registros de las
## celdas disponibles que esten conectadas entre si
def _flow(isOpen, isFull, i, j):
    n = len(isFull)
    if (i < 0) or (i >= n):
        return
    if (j < 0) or (j >= n):
        return
    if (isOpen[i][j] == 0):
        return
    if (isFull[i][j] == 1):
        return
    isFull[i][j] = 1
    _flow(isOpen, isFull, i+1, j  )  ## Abajo
    _flow(isOpen, isFull, i  , j+1)  ## Derecha
    _flow(isOpen, isFull, i  , j-1)  ## Izquierda
    _flow(isOpen, isFull, i-1, j  )  ## Arriba

In [ ]:
## recibe n y m que son valores para las dimensiones de la matriz
def main(n, probabilidadDisponible):
    array = armarArray(n, probabilidadDisponible)
    print(array)
    print(percola(array))
    
        

In [ ]:
## Ejecuta el metodo main que toma las dimensiones n*n de la matriz 
## y un numero float entre 0 y 1 el cual representara la 
## probabilidad de asumir 1 o disponible
main(5, 0.5)